In [1]:
import os
import pandas as pd
import geopandas as gpd
from spatial_access.Models import AccessCount, AccessModel, AccessSum, AccessTime

### Prepare data

- https://docs.google.com/document/d/1QAiBs0egb6Y4eGHyExEOjxItZORgq-yxKlhKs2j5d3Q/edit?ts=5f161005

In [2]:
dist_lookup = {'Methadone': {'Urban': 2, 'Rural': 10},\
              'Buprenorphine': {'Urban': 5, 'Rural': 20},\
              'Naltrexone': {'Urban': 5, 'Rural': 20}}

- Zip code to latitude and longitude

In [19]:
df2003_MM = pd.read_csv("../2003/df2003_MM.csv", dtype={'ZIP_Code':str})
df2003_MOUD = pd.read_csv("../2003/2003.csv", dtype={'ZIP_Code':str})

### Calculate access metrics
- https://github.com/GeoDaCenter/spatial_access/blob/master/docs/notebooks/4_Access_Metrics.ipynb
- https://github.com/GeoDaCenter/spatial_access/blob/master/docs/notebooks/spatial_access_documentation081219.pdf
- https://drive.google.com/drive/folders/1RKOxpnwSkXp_nNA7Z2bK7EYjI5DRnNTN

In [5]:
#location of GEOIDs
origins = pd.read_csv('all_zip_centroids.csv', dtype={'GEOID':str})
origins

,GEOID,Y,X,POP,state
0,99929,56.393672,-132.314789,2338,2
1,99927,56.307593,-133.582620,94,2
2,99926,55.121319,-131.573811,1460,2
3,99925,55.554289,-133.077745,819,2
4,99923,55.935375,-130.037812,87,2
...,...,...,...,...,...
32931,01007,42.272966,-72.409206,14649,25
32932,01005,42.411222,-72.110303,5079,25
32933,01003,42.389133,-72.524682,10372,25
32934,01002,42.372941,-72.511333,29049,25


In [14]:
origins[origins['GEOID'] == '60617']
#seems to be the ZIP codes...?

,GEOID,Y,X,POP,state
12409,60617,41.723948,-87.555017,84155,17


In [20]:
#location of MOUDs
df2003_MOUD['ID'] = list(df2003_MOUD.index + 1)
df2003_MOUD

,Year,Page,Center_name,Street_address,City,State,ZIP_Code,Phone,Keys,ID
0,2003,15,Lighthouse of Tallapoosa County Inc Substance ...,36 Franklin Street,Alexander City,AL,35010,(256) 234-4894,SA TX HH RR MN SFANNISTON,1
1,2003,15,Anniston Fellowship House Inc,106 East 22nd Street,Anniston,AL,36201,(256) 236-7229,SA TX HH RR MN SF AH,2
2,2003,15,Calhoun/Cleburne Mental Health Center New Dire...,407 South Noble Street,Anniston,AL,36201,(256) 236-8003 Hotlines: (256) 236-8003 (256) ...,SA TX OP AD CO DU CJ SF MD PI MI SSASHVILLE,3
3,2003,15,Bridge Inc Recovery Center for Teens/Ashville,474 10th Street,Ashville,AL,35953,(205) 594-3111 www.bridgeinc.org,SA TX OP AD COBESSEMER,4
4,2003,15,Northwest Alabama Treatment Center,709 Memorial Drive,Bessemer,AL,35020,(205) 425-1200 Methadone,SA TX DT MM DM OP CO GL SE PW WO SF AHBIRMINGHAM,5
...,...,...,...,...,...,...,...,...,...,...
11563,2003,741,Wyoming Substance Abuse Treatment and Recovery...,1003 Saberton Street,Sheridan,WY,82801,(307) 672-2044 wystarrecover.com,SA TX OP RR CO SF PI MI SS PATHERMOPOLIS,11564
11564,2003,741,Hot Springs County Counseling Service,121 South 4th Street,Thermopolis,WY,82443,(307) 864-3138,MH TX OP CO DU SF MD PI MITORRINGTON,11565
11565,2003,741,Southeast Wyoming Mental Health Center Goshen ...,501 Albany Avenue,Torrington,WY,82240,(307) 532-4091 Hotline: (307) 532-4091 www.sew...,MH-SA TX OP AD SF MD MC PI MI SS SP AHWHEATLAND,11566
11566,2003,741,Southeast Wyoming Mental Health Center Substan...,103 Park Avenue,Wheatland,WY,82201,(307) 322-3190,MH TX OP DU CJ SF MD MC PI SS AHWORLAND,11567


In [23]:
#make MOUDs a geodataframe by zipcode locations
dests_df = pd.merge(df2003_MOUD, origins, how='left', left_on='ZIP_Code', right_on='GEOID')

In [26]:
dests = gpd.GeoDataFrame(dests_df, geometry = gpd.points_from_xy(dests_df.X, dests_df.Y))
dests

,Year,Page,Center_name,Street_address,City,State,ZIP_Code,Phone,Keys,ID,GEOID,Y,X,POP,state,geometry
0,2003,15,Lighthouse of Tallapoosa County Inc Substance ...,36 Franklin Street,Alexander City,AL,35010,(256) 234-4894,SA TX HH RR MN SFANNISTON,1,35010,32.932861,-85.946004,20816.0,1.0,POINT (-85.94600 32.93286)
1,2003,15,Anniston Fellowship House Inc,106 East 22nd Street,Anniston,AL,36201,(256) 236-7229,SA TX HH RR MN SF AH,2,36201,33.659047,-85.861684,20156.0,1.0,POINT (-85.86168 33.65905)
2,2003,15,Calhoun/Cleburne Mental Health Center New Dire...,407 South Noble Street,Anniston,AL,36201,(256) 236-8003 Hotlines: (256) 236-8003 (256) ...,SA TX OP AD CO DU CJ SF MD PI MI SSASHVILLE,3,36201,33.659047,-85.861684,20156.0,1.0,POINT (-85.86168 33.65905)
3,2003,15,Bridge Inc Recovery Center for Teens/Ashville,474 10th Street,Ashville,AL,35953,(205) 594-3111 www.bridgeinc.org,SA TX OP AD COBESSEMER,4,35953,33.817182,-86.242649,7319.0,1.0,POINT (-86.24265 33.81718)
4,2003,15,Northwest Alabama Treatment Center,709 Memorial Drive,Bessemer,AL,35020,(205) 425-1200 Methadone,SA TX DT MM DM OP CO GL SE PW WO SF AHBIRMINGHAM,5,35020,33.404184,-86.952464,27139.0,1.0,POINT (-86.95246 33.40418)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11563,2003,741,Wyoming Substance Abuse Treatment and Recovery...,1003 Saberton Street,Sheridan,WY,82801,(307) 672-2044 wystarrecover.com,SA TX OP RR CO SF PI MI SS PATHERMOPOLIS,11564,82801,44.784705,-106.963663,24546.0,56.0,POINT (-106.96366 44.78471)
11564,2003,741,Hot Springs County Counseling Service,121 South 4th Street,Thermopolis,WY,82443,(307) 864-3138,MH TX OP CO DU SF MD PI MITORRINGTON,11565,82443,43.668971,-108.254914,4697.0,56.0,POINT (-108.25491 43.66897)
11565,2003,741,Southeast Wyoming Mental Health Center Goshen ...,501 Albany Avenue,Torrington,WY,82240,(307) 532-4091 Hotline: (307) 532-4091 www.sew...,MH-SA TX OP AD SF MD MC PI MI SS SP AHWHEATLAND,11566,82240,42.066975,-104.184360,10235.0,56.0,POINT (-104.18436 42.06698)
11566,2003,741,Southeast Wyoming Mental Health Center Substan...,103 Park Avenue,Wheatland,WY,82201,(307) 322-3190,MH TX OP DU CJ SF MD MC PI SS AHWORLAND,11567,82201,42.050198,-104.987311,6467.0,56.0,POINT (-104.98731 42.05020)


In [8]:
#map of zipcodes
us_zips = gpd.read_file('tl_2010_us_zcta510/tl_2010_us_zcta510.shp')
us_zips.geometry = us_zips.geometry.to_crs(epsg = 4326)
us_zips

,ZCTA5CE10,GEOID10,CLASSFP10,MTFCC10,FUNCSTAT10,ALAND10,AWATER10,INTPTLAT10,INTPTLON10,geometry
0,04757,04757,B5,G6350,S,334700805,3663844,+46.6676144,-068.1686355,"POLYGON ((-68.08095 46.56746, -68.08236 46.567..."
1,04758,04758,B5,G6350,S,91006829,191946,+46.5592719,-067.8532254,"POLYGON ((-67.90650 46.52526, -67.90677 46.533..."
2,04760,04760,B5,G6350,S,259787771,547534,+46.3393412,-067.9548745,"POLYGON ((-67.85272 46.37207, -67.84753 46.372..."
3,04975,04975,B5,G6350,S,142318,0,+44.6253540,-069.5883338,"POLYGON ((-69.58945 44.62188, -69.58938 44.622..."
4,04976,04976,B5,G6350,S,257908035,4722278,+44.7892800,-069.6676898,"POLYGON ((-69.73503 44.78785, -69.73388 44.787..."
...,...,...,...,...,...,...,...,...,...,...
33115,05456,05456,B5,G6350,S,62050717,2507592,+44.2214160,-073.2645956,"POLYGON ((-73.32482 44.22666, -73.32495 44.226..."
33116,05457,05457,B5,G6350,S,99545684,6083881,+44.9576932,-072.9179234,"POLYGON ((-72.96983 44.92554, -72.96914 44.925..."
33117,05458,05458,B5,G6350,S,42417581,39322,+44.7178468,-073.3074859,"MULTIPOLYGON (((-73.34469 44.74012, -73.34498 ..."
33118,05459,05459,B5,G6350,S,66399633,1290552,+44.9500605,-073.0053668,"POLYGON ((-72.98310 44.97364, -72.98263 44.971..."


In [9]:
#join map and MOUDs
joined = gpd.sjoin(df2003_MOUD, us_zips[['GEOID10','geometry']], how='left', op='intersects')
joined['X'] = joined.geometry.centroid.x
joined['Y'] = joined.geometry.centroid.y
joined = joined[~joined.GEOID10.isna()]
joined.to_csv('destinations.csv', index=False)

ValueError: 'left_df' should be GeoDataFrame, got <class 'pandas.core.frame.DataFrame'>

In [3]:
accessM = AccessModel(network_type = 'drive',
                     transit_matrix_filename = './m_full_cost_matrix.csv',
                     sources_filename = './pop_weighted_centroids_study_area.csv',
                     destinations_filename = './moud_subset.csv',
                     source_column_names = source_col_names,
                     dest_column_names = dest_col_names,
                     decay_function= 'linear') # distance decay where closer amenities have more weight 